In [1]:
import json
import pandas as pd
import numpy as np
import re
import sqlalchemy
from sqlalchemy import create_engine
from config import db_password
import time

In [239]:
# Create a function that takes in three arguments

def etl_auto(wiki, kaggle):
    file_dir = 'C:/Users/shula/OneDrive/Desktop/UCB_Data_052020/UCB-202005-downloads/data'
    
    # Extract data from multiple sources 
    
    try:
        kaggle_df = pd.read_csv(f'{file_dir}/{kaggle}.csv', low_memory = False) 
        print('kaggle data converted')
        #rating_df = pd.read_csv(f'{file_dir}/{rating}.csv', low_memory = False)       
        #print('rating data converted')
        wiki_df = pd.read_csv(f'{file_dir}/{wiki}.csv', low_memory = False)   
        print('wiki data converted')
    except FileNotFoundError:    
            with open(f'{file_dir}/{wiki}.json', mode='r') as f:
                raw = json.load(f)
            pass
    
    # Clean wiki data and convert into dataframe 
    
    wiki_movies = [movie for movie in raw
                   if ('Director' in movie or 'Directed by' in movie) 
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
    
    def clean_movie(movie):
        alt_titles ={}
        # Combine altenative titles into one list 'alt_titles' 
        for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                    'Hangul','Hebrew','Hepburn','Japanese','Literally',
                    'Mandarin','McCune–Reischauer','Original title','Polish',
                    'Revised Romanization','Romanized','Russian',
                    'Simplified','Traditional','Yiddish']:
            if key in movie:
                alt_titles[key] = movie[key]
                movie.pop(key)
        if len(alt_titles) > 0:
            movie['alt_titles'] = alt_titles
   
    # Merge similar columns name
        def change_column_name(old_name, new_name):
            if old_name in movie:
                movie[new_name] = movie.pop(old_name)
        change_column_name('Adaptation by', 'Writer(s)')
        change_column_name('Country of origin', 'Country')
        change_column_name('Directed by', 'Director')
        change_column_name('Distributed by', 'Distributor')
        change_column_name('Edited by', 'Editor(s)')
        change_column_name('Length', 'Running time')
        change_column_name('Original release', 'Release date')
        change_column_name('Music by', 'Composer(s)')
        change_column_name('Produced by', 'Producer(s)')
        change_column_name('Producer', 'Producer(s)')
        change_column_name('Productioncompanies ', 'Production company(s)')
        change_column_name('Productioncompany ', 'Production company(s)')
        change_column_name('Released', 'Release Date')
        change_column_name('Release Date', 'Release date')
        change_column_name('Screen story by', 'Writer(s)')
        change_column_name('Screenplay by', 'Writer(s)')
        change_column_name('Story by', 'Writer(s)')
        change_column_name('Theme music composer', 'Composer(s)')
        change_column_name('Written by', 'Writer(s)')
    
        # Return output
        return movie
        
    
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
    wiki_movies_df = pd.DataFrame(clean_movies)
    print('wiki data converted')

In [240]:
etl_auto('wikipedia_movies','movies_metadata')

kaggle data converted
wiki data converted
